In [1]:
from flask import Flask, render_template, request, jsonify , make_response , send_file
import re 
import ast
from flask_cors import CORS
import pandas as pd
app = Flask(__name__)
CORS(app)


In [2]:
from groq import Groq

client = Groq(
    api_key="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",
)
df = []
tst = ""
info = ""

In [3]:
import numpy as np
import matplotlib.pyplot as plt
cnt=1

In [4]:
def search_dataset(message) :
    # Search DATASET ...

    chat_completion = client.chat.completions.create(
    messages=[

        {
            "role": "system",
            "content": "You are a Professional Data Scientist , Search for top 5 links from kaggle to download the dataset requested by the user , get some somewhere else if it is not in Kaggle , be precise" ,
        },
        {
            "role": "user",
            "content": f"{message}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    print(response_text)
    return response_text


In [5]:
def preprocess(mess) :

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a Professional Data Scientist , Write a python code to complete the requested task , do all in inplace so that it updates in realtime" ,
            },
            {
                "role": "user", 
                "content": f"This is the dataset i have : {tst} and it is loaded in a variable named as df, give a code to {mess} also call it  , give code only dont give explanation " ,
            }
        ],
        model="llama3-70b-8192",
        )
    
    # akn = client.chat.completions.create(
    #     messages=[
    #         {
    #     "role": "user",
    #     "content": "Write a success aknowledgement message in 3 lines for {mess}" ,
    # }
    #     ],
    #     model="llama3-70b-8192",
    #     )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    
    print(viss)
    pattern = r"```(.*?)```"
    local_context = {}  
    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)
    pattern12 = r"$$(.*?)$$"
    matches12 = re.findall(pattern12, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])
    result = ""
    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    # result_value = local_context.get('result')

    return "matches12[0]",result


In [6]:
@app.route('/process_file', methods=['POST'])
def process_file():
    global info 
    global df
    global tst 
    file = request.files['file']
    if file:
        file.save('data.csv')
        df = pd.read_csv('data.csv')
        tst = df.head()
        info = df.info()
        chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the dataset i have : {tst} , explain and overview of it in 2 lines" ,
            }
        ],
        model="llama3-70b-8192",
        )
        response_text = chat_completion.choices[0].message.content
        print(response_text)
        return jsonify({'response': response_text})
    else:
        return jsonify({'response': 'No file provided'})




In [25]:
# # # Process CSV 
# # @app.route('/process_data', methods=['POST'])

# def process_csv () :
#     global df
#     path = ""
#     df = pd.read_csv(path)
#     chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": f"This is the dataset i have : {tst} , explain and summarize it in 2 lines" ,
#         }
#     ],
#     model="llama3-8b-8192",
#     )
#     response_text = chat_completion.choices[0].message.content
#     print(response_text)


In [7]:
def suggest() :
    # Suggessions

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"I'm working on preprocessing a DataSet ,  This is the dataset i have : {tst} and its info : {info},give me 3 suggests to what to do further in 3 lines" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

In [8]:
def others(mess) :

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"You are a AI data analyst , answer the below question according to your role , QUESTION : {mess}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text

In [9]:
cnt=1

In [10]:
def visualize(mess) :
    # Visualize 
    global cnt
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            # "content": f"This is the dataset i have : {tst} , give a code to visualize column1 and column2 also call it ,mention the column names exactly , give code only dont give explanation " ,
            "content": f"This is the dataset i have : {tst} , give a code to visualize {mess}using matplotlib , also call it ,give the column names exactly as in the dataset give clean code also save the plot as 'plot{cnt}.png' "
        }
    ],
    model="llama3-70b-8192",
    )
    viss = chat_completion.choices[0].message.content
    viss =  re.sub('python', '', viss)
    viss =  re.sub('Python', '', viss)
    pattern = r"```(.*?)```"

    # Find all matches
    matches = re.findall(pattern, viss, re.DOTALL)

    # Safely parse the response text as a Python AST (Abstract Syntax Tree)
    parsed_ast = ast.parse(matches[0])

    # else:
    # Execute the parsed AST
    exec(compile(parsed_ast, filename="<ast>", mode="exec"))
    
    return f"C:/Prodigy/DataGenie/plot{cnt}.png"



In [11]:
# def classify(mess) :
    
#     chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": f"""You are an assistant with knowledge on Data Science who classifies user prompt into one of the following categories:
#               visualization : if user need to visualize the data using any plots or charts,
#               searching : If the user is searcing for dataset available in online or he wants any dataset to be searched ,
#               data_cleaning : If the user wants to do any cleaning or preprocessing tasks ,
#               model_training : If the user wants to train a Model ,
#               suggestions : if the user wants to get some suggessions to what to do with the dataset  ,
#               Quering_dataset : If the user whats to check what is in the the dataset like unique values , datatypes , df.head() , or df.tail() . 
              
#               \n\n Don't give any kind of Explanation just give the classification """ ,
#         },
#         {
#             "role": "user",
#             "content": f"{mess}" ,
#         }
#     ],
#     model="llama3-70b-8192",
#     )
#     response_text = chat_completion.choices[0].message.content
#     return response_text

In [11]:

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/send_message', methods=['POST'])
def send_message():
    global cnt
    message = request.get_json()['message']
    cls = classify(message)
    print(cls)
    if "visualization" in cls :
        res = visualize(message)
        cnt += 1
        return jsonify({'response': "image" , "path" : res})
    elif "searching" in cls :
        res = search_dataset(message)

    elif 'data_cleaning' in cls or 'Quering_dataset' in cls :
        akg,msg = preprocess(message)
        res = akg+'\n\n' +msg
        # return res
    elif 'suggestions' in cls :
        res = suggest()
        # return res
    elif 'model_training' in cls :
        pass
        # res = suggest()
        # return res
    else :
        res = others(message)

    print(res)
    # if "visualize" in message.lower() or "plot" in message.lower() :
    #     res = visualize(message)
    #     cnt += 1
    #     return jsonify({'response': "image" , "path" : res})
    # response = process_message(message)
    return jsonify({'response': res })


def classify(mess) :
    
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"""You are an assistant with knowledge on Data Science who classifies user prompt into one of the following categories:
              visualization : if user need to visualize the data using any plots or charts,
              searching : If the user is searcing for dataset available in online or he wants any dataset to be searched ,
              data_cleaning : If the user wants to do any cleaning or preprocessing tasks ,
              model_training : If the user wants to train a Model ,
              suggestions : if the user wants to get some suggessions to what to do with the dataset  ,
              Quering_dataset : If the user whats to check what is in the the dataset like unique values , datatypes , df.head() , or df.tail() , 
              Others : If the query doesn't falls below any of the above classes like Hi , Hello and more .

              \n\n Don't give any kind of Explanation just give the classification """ ,
        },
        {
            "role": "user",
            "content": f"{mess}" ,
        }
    ],
    model="llama3-70b-8192",
    )
    response_text = chat_completion.choices[0].message.content
    return response_text
def process_message(message):
    # Implement your chatbot logic here
    res = ""
    if "search a dataset" in message.lower() :
        res = search_dataset(message)
        # return res

        # return res
    if 'preprocess' in message.lower() or 'clean' in message.lower() :
        res = preprocess(message)
        # return res
    if 'suggest' in message.lower() or 'next' in message.lower() :
        res = ()
        # return res
    else :
        res = others(message)
        
    return res
    return "image"


In [12]:
@app.route('/download', methods=['GET'])
def download_csv():
    # Convert the DataFrame to a CSV string
    csv_string = df.to_csv(index=False)

    # Set the response headers
    response = make_response(csv_string)
    response.headers['Content-Disposition'] = 'attachment; filename="data.csv"'
    response.headers['Content-Type'] = 'text/csv'

    return response

In [13]:
if __name__ == '__main__':
    app.run(port=7000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [15/Jul/2024 21:33:50] "OPTIONS /send_message HTTP/1.1" 200 -


Others


127.0.0.1 - - [15/Jul/2024 21:33:51] "POST /send_message HTTP/1.1" 200 -


Hello! As a data analyst, I'm happy to help you with any questions or insights you'd like to gain from your data. What kind of data are you working with, and what would you like to explore or understand better? Are you looking for trends, patterns, or correlations? Or perhaps you have a specific question in mind? Let me know, and I'll do my best to assist you!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float6

...

[2024-07-15 21:34:18,546] ERROR in app: Exception on /send_message [POST]
Traceback (most recent call last):
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask_cors\extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^

Here is the  code to handle null values and update the dataframe in real-time:
```
import pandas as pd

# assume df is the dataframe

# check for null values
print(df.isnull().sum())

# fill null values with appropriate values (e.g. mean, median, etc.)
df.fillna(df.mean(), inplace=True)

# or fill null values with a specific value
# df.fillna(0, inplace=True)

# update the dataframe in real-time
print(df.head())


127.0.0.1 - - [15/Jul/2024 21:34:47] "OPTIONS /send_message HTTP/1.1" 200 -


visualization


[2024-07-15 21:34:49,095] ERROR in app: Exception on /send_message [POST]
Traceback (most recent call last):
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask_cors\extension.py", line 178, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\BARATH RAJ P\AppData\Local\Programs\Python\Python312\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^